In [1]:
import pandas as pd
import sqlite3
import numpy as np
import random
import datetime
#import matplotlib as plt
from matplotlib import pyplot as plt

In [19]:
def fix_dividends(symbol, offset, df):
    '''
    Used to obtain dividend per quater using dividends, YTD
    Adds the column 'div' and returns the new dataframe
    symbol: the ticker symbol being operated on
    offset: the quarter of the first row of data represented as [0,1,2,3]
    df: a dataframe contianing at least ['Dividends Paid Per Share (YTD)']    
    '''
    for index, row in df.iterrows():
        #if index > 3:
        #    df.at[index,'eps_last_four_quarters'] = df.at[index, 'Basic EPS (Continuing)'] + df_earn.at[index-1, 'Basic EPS (Continuing)'] + df_earn.at[index-2, 'Basic EPS (Continuing)'] + df_earn.at[index-3, 'Basic EPS (Continuing)'] 
        #    df.at[index,'div_last_four_quarters'] = df.at[index, 'div'] + df.at[index-1, 'div'] + df_earn.at[index-2, 'div'] + df_earn.at[index-3, 'div']

        if index % 4 == offset:
            df.at[index,'_div'] = df.at[index,'Dividends Paid Per Share (YTD)']
        else:
            df.at[index,'_div'] = df.at[index,'Dividends Paid Per Share (YTD)'] - df.at[index -1,'Dividends Paid Per Share (YTD)']

    return df

def update_dividends(symbol, db, df):
    '''
    commits the div column to SQLite
    symbol = ticker symbol to write
    db = the database    
    '''
    cur = db.cursor()
    
    for index, row in df.iterrows():
        
        sql_string = ''' UPDATE quarterly_reports SET _div = ? WHERE company_ticker = ? AND report_date = ?'''
        task = (row['div'], symbol, row['report_date'])
        cur.execute(sql_string, task)
        
    db.commit()
    
    
    
    
    

In [7]:
#db_yahoo = '/home/ian/Data/yahoo.db'
#database_yahoo = sqlite3.connect(sqlite_db)
#yahoo_string = '''SELECT * FROM aav_prices WHERE symbol = 'BNS' UNION ALL SELECT * FROM tsx_prices WHERE symbol = 'BNS';'''

#df_yahoo = pd.read_sql_query(yahoo_string, database_yahoo)
                       
db_advfn = '/home/ian/Data/advfn.db'
database_advfn = sqlite3.connect(db_advfn)

advfn_string = '''SELECT * FROM quarterly_reports WHERE company_ticker = 'BNS';'''

df_earn = pd.read_sql_query(advfn_string, database_advfn)

                       

In [14]:
df_earn = fix_dividends('BNS', 0, df_earn)

In [20]:
update_dividends('BNS', database_advfn, df_earn)

In [16]:
df_earn[['report_date','Dividends Paid Per Share (YTD)', 'div']]

report_date  Dividends Paid Per Share (YTD)   div
0      1996/01                            0.08  0.08
1      1996/04                            0.16  0.08
2      1996/07                            0.24  0.08
3      1996/10                            0.33  0.09
4      1997/01                            0.10  0.10
5      1997/04                            0.19  0.09
6      1997/07                            0.28  0.09
7      1997/10                            0.37  0.09
8      1998/01                            0.10  0.10
9      1998/04                            0.20  0.10
10     1998/07                            0.30  0.10
11     1998/10                            0.40  0.10
12     1999/01                            0.11  0.11
13     1999/04                            0.21  0.10
14     1999/07                            0.21  0.00
15     1999/10                            0.21  0.00
16     2000/01                            0.00  0.00
17     2000/04                            0.24  0.24
18     2000/07                            0.24  0.00
19     2000/10                            0.24  0.00
20     2001/01                            0.00  0.00
21     2001/04                            0.30  0.30
22     2001/07                            0.45  0.15
23     2001/10                            0.62  0.17
24     2002/01                            0.00  0.00
25     2002/04                            0.19  0.19
26     2002/07                            0.37  0.18
27     2002/10                            0.56  0.19
28     2003/01                            0.20  0.20
29     2003/04                            0.49  0.29
..         ...                             ...   ...
63     2011/10                            2.05  0.52
64     2012/01                            0.52  0.52
65     2012/04                            1.07  0.55
66     2012/07                            1.62  0.55
67     2012/10                            2.19  0.57
68     2013/01                            0.57  0.57
69     2013/04                            1.17  0.60
70     2013/07                            1.77  0.60
71     2013/10                            2.39  0.62
72     2014/01                            0.62  0.62
73     2014/04                            1.26  0.64
74     2014/07                            1.90  0.64
75     2014/10                            2.56  0.66
76     2015/01                            0.66  0.66
77     2015/04                            1.34  0.68
78     2015/07                            2.70  1.36
79     2015/10                            3.38  0.68
80     2016/01                            0.70  0.70
81     2016/04                            1.42  0.72
82     2016/07                            2.14  0.72
83     2016/10                            2.88  0.74
84     2017/01                            0.74  0.74
85     2017/04                            1.50  0.76
86     2017/07                            2.26  0.76
87     2017/10                            3.05  0.79
88     2018/01                            0.79  0.79
89     2018/04                            1.61  0.82
90     2018/07                            2.43  0.82
91     2018/10                            3.28  0.85
92     2019/01                            0.85  0.85

[93 rows x 3 columns]

In [ ]:
### SCRAP

offset = 0

df_earn['eps_last_four_quarters'] = 0.0
df_earn['div'] = 0.0
df_earn['div_last_four_quarters'] = 0.0

for index, row in df_earn.iterrows():
    if index > 3:
        df_earn.at[index,'eps_last_four_quarters'] = df_earn.at[index, 'Basic EPS (Continuing)'] + df_earn.at[index-1, 'Basic EPS (Continuing)'] + df_earn.at[index-2, 'Basic EPS (Continuing)'] + df_earn.at[index-3, 'Basic EPS (Continuing)'] 
        df_earn.at[index,'div_last_four_quarters'] = df_earn.at[index, 'div'] + df_earn.at[index-1, 'div'] + df_earn.at[index-2, 'div'] + df_earn.at[index-3, 'div']
        
    if index % 4 == offset:
        df_earn.at[index,'div'] = df_earn.at[index,'Dividends Paid Per Share (YTD)']
    else:
        df_earn.at[index,'div'] = df_earn.at[index,'Dividends Paid Per Share (YTD)'] - df_earn.at[index -1,'Dividends Paid Per Share (YTD)']
